<a href="https://colab.research.google.com/github/Liam876/t-SNE_Project/blob/main/notebooks/Simple_Classifiers_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load data

In [ ]:
from time import time
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import NullFormatter
from sklearn import datasets, manifold
from sklearn.decomposition import PCA
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D, Reshape
from tensorflow.keras.regularizers import l2

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading the data

In [ ]:
import pandas as pd

path_to_X_init1 ="/content/drive/MyDrive/tsne_circles/Circles_csvs/X_init1"
path_to_X_ex1 =  "/content/drive/MyDrive/tsne_circles/Circles_csvs/X_ex1"
path_to_y1 = "/content/drive/MyDrive/tsne_circles/Circles_csvs/y1"
X_init1 = pd.read_csv(path_to_X_init1, header= None).values
X_ex1 = pd.read_csv(path_to_X_ex1, header= None).values
y1 = pd.read_csv(path_to_y1, header= None).values

path_to_X_init2 ="/content/drive/MyDrive/tsne_circles/Circles_csvs/X_init2"
path_to_X_ex2 =  "/content/drive/MyDrive/tsne_circles/Circles_csvs/X_ex2"
path_to_y2 = "/content/drive/MyDrive/tsne_circles/Circles_csvs/y2"
X_init2 = pd.read_csv(path_to_X_init2, header= None).values
X_ex2 = pd.read_csv(path_to_X_ex2, header= None).values
y2 = pd.read_csv(path_to_y2, header= None).values

path_to_X_init3 ="/content/drive/MyDrive/tsne_circles/Circles_csvs/X_init3"
path_to_X_ex3 =  "/content/drive/MyDrive/tsne_circles/Circles_csvs/X_ex3"
path_to_y3 = "/content/drive/MyDrive/tsne_circles/Circles_csvs/y3"
X_init3 = pd.read_csv(path_to_X_init3, header= None).values
X_ex3 = pd.read_csv(path_to_X_ex3, header= None).values
y3 = pd.read_csv(path_to_y3, header= None).values

In [ ]:
X_init = np.vstack((X_init1,X_init2,X_init3))
X_ex = np.vstack((X_ex1,X_ex2,X_ex3))
y = np.concatenate((y1,y2,y3))

In [ ]:
X_ex.shape

(15000, 300)

In [ ]:
print(f"{round(np.sum(y)*100/len(y),2)}% are 1-labeled examples")

44.23% are 1-labeled examples


#Models

##Feedforward

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_ex, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Dense(units=300, input_dim=300, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(units=64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with a lower learning rate
#custom_optimizer = Adam(learning_rate=10**-3)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


##Conv

In [ ]:
def create_model():

    input_shape = (300,)

    model = Sequential([
    # Reshape the 1D input into a 2D input
    Reshape((75, 4), input_shape=input_shape),

    # Convolutional layer with 64 filters and a kernel size of 3
    Conv1D(64, kernel_size=25, activation='relu'),
    #MaxPooling1D(pool_size=2),

    # Convolutional layer with 128 filters and a kernel size of 3
    Conv1D(128, kernel_size=3, activation='relu'),
    #MaxPooling1D(pool_size=2),

    # Convolutional layer with 256 filters and a kernel size of 3
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),

    # Flatten the output to connect to dense layers
    Flatten(),

    # Dense layer with 256 units, ReLU activation, and dropout for regularization
    Dense(256, activation='relu'),
    Dropout(0.5),

    # # Dense layer with 128 units and ReLU activation
    # Dense(128, activation='relu'),

    # Output layer with a single neuron for binary classification
    Dense(1, activation='sigmoid')
])

    return model

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Assuming X and y are defined

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_ex, y, test_size=0.2, random_state=42)
model = create_model()

# Compile the model with a lower learning rate
custom_optimizer = Adam(learning_rate=1e-3)
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


##Distribution- Based

In [ ]:
#preparing the data and creating constants
n_samples = 100
start = 900
X_data = X_init[start:start+n_samples]
y_data = y[start:start+n_samples]
n_points = int(X_data.shape[1]/2)
n_pairs = int((n_points**2 - n_points)/2)

#Calculate the Cauchy similarity between 2 points
def cauchy_similarity(x,y,sigma=1.0):
  dist = np.linalg.norm(x-y)
  sim = 1/(1+ (dist/sigma))
  return sim

#Transform the points into distribution
def transform_sample(sample):
  #print(type(sample))
  #print(sample)
  points = sample.reshape((n_points,2))
  res = np.zeros((n_pairs))
  part_fun = 0
  idx = 0

  for i in range(n_points):
    for j in range(i+1,n_points):
      sim = cauchy_similarity(points[i],points[j])
      res[idx] = sim
      part_fun += sim
      idx +=1

  assert res[-1] != 0 # Just to make sure the list is full
  res = res/part_fun # Normalize the distribution
  return res

# Generator function for on-the-fly data transformation
def data_generator(X, y, batch_size=32):
    while True:
        indices = np.random.choice(len(X), batch_size, replace=False)
        batch_X = X[indices]
        batch_y = y[indices]

        # Perform on-the-fly transformation for each sample in the batch
        transformed_batch_X = np.array([transform_sample(sample) for sample in batch_X])

        yield transformed_batch_X, batch_y
class TopKMaskingLayer(tf.keras.layers.Layer):
    def __init__(self, k, **kwargs):
        super(TopKMaskingLayer, self).__init__(**kwargs)
        self.k = k

    def call(self, inputs):
        top_k_values, top_k_indices = tf.nn.top_k(inputs, k=self.k)
        #mask = tf.cast(tf.math.less(tf.range(tf.shape(inputs)[-1])[:, tf.newaxis], top_k_indices), dtype=tf.float32)
        return top_k_values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
validation_data = np.array([transform_sample(s) for s in X_test]).reshape((X_test.shape[0],-1))
# Model definition

class MaxThresholdLayer(tf.keras.layers.Layer):
    def __init__(self, threshold_value, **kwargs):
        super(MaxThresholdLayer, self).__init__(**kwargs)
        self.threshold_value = threshold_value

    def call(self, inputs):
        return tf.maximum(inputs, self.threshold_value)

max_threshold_layer = tf.keras.layers.Lambda(lambda x: tf.maximum(x, 0.01), input_shape=(n_pairs,))

model = tf.keras.Sequential([
    #TopKMaskingLayer(k=1000, input_shape=(n_pairs,)),
    tf.keras.layers.Dense(64, activation='relu',input_shape=(n_pairs,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using the generator
batch_size = 32
steps_per_epoch = len(X_train) // batch_size

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

model.fit(data_generator(X_train, y_train, batch_size),
          epochs=500,
          steps_per_epoch=steps_per_epoch,
          validation_data=(validation_data, y_test),
           callbacks=[early_stopping])

# Evaluate the model
y_pred = model.predict(validation_data)
y_pred_binary = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')